Establish drive file stream:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Imports:

In [ ]:
!pip install "ray[tune]"

import os
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.autograd import Variable
import time
import copy
import numpy as np
from torchvision import datasets, models, transforms
from PIL import Image
import torch.nn.functional as F
import seaborn as sn
import pandas as pd
from sklearn.metrics import f1_score
from graphviz import Digraph
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
import random

Defining dataset:

In [ ]:
class PatientDataset(Dataset):

  def __init__(self, sourceFolder, input_size, phase):
    self.patients = []
    self.classes = []
    self.classIdx = []
    for dir in os.listdir(sourceFolder):
      if '.' not in dir:
        self.patients.append(os.path.join(sourceFolder, dir))
        label = dir.split('_')[0]
        classFound = False
        for i in range(len(self.classes)):
          if label in self.classes[i]:
            self.classIdx.append(i)
            classFound = True
            break
        if not classFound:
          # print('New class found: ' + label)
          self.classes.append(label)
          self.classIdx.append(len(self.classes) - 1)
    self.patientSamples = []
    self.cntPerClass = [0] * len(self.classes)
    for i in range(len(self.classIdx)):
      self.cntPerClass[self.classIdx[i]] += 1
      self.patientSamples.append((self.patients[i], self.classIdx[i]))
    print(self.classes)
    self.weightsPerClasses = [0] * len(self.classes)
    for i in range(len(self.cntPerClass)):
      self.weightsPerClasses[i] = 1 / (self.cntPerClass[i] / len(self.classIdx))
    print(self.weightsPerClasses)


    if 'train' in phase:
          self.SATransforms = transforms.Compose([
                transforms.Grayscale(),
                transforms.CenterCrop(input_size),
                transforms.RandomRotation(10),
                transforms.RandomPerspective(0.1),
                transforms.ToTensor(),
                transforms.Normalize([0.1858], [0.1787])]
          )
          self.CH2Transforms = transforms.Compose([
              transforms.Grayscale(),
              transforms.CenterCrop(input_size),
              transforms.RandomRotation(10),
              transforms.RandomPerspective(0.1),
              transforms.ToTensor(),
              transforms.Normalize([0.1635], [0.2076])]
          )
          self.CH3Transforms = transforms.Compose([
              transforms.Grayscale(),
              transforms.CenterCrop(input_size),
              transforms.RandomRotation(10),
              transforms.RandomPerspective(0.1),
              transforms.ToTensor(),
              transforms.Normalize([0.1613], [0.2128])]
          )
          self.CH4Transforms = transforms.Compose([
              transforms.Grayscale(),
              transforms.CenterCrop(input_size),
              transforms.RandomRotation(10),
              transforms.RandomPerspective(0.1),
              transforms.ToTensor(),
              transforms.Normalize([0.1683], [0.2181])]
          )
    else:
          self.SATransforms = transforms.Compose([
                transforms.Grayscale(),
                transforms.CenterCrop(input_size),
                transforms.ToTensor(),
                transforms.Normalize([0.1858], [0.1787])]
          )
          self.CH2Transforms = transforms.Compose([
                transforms.Grayscale(),
                transforms.CenterCrop(input_size),
                transforms.ToTensor(),
                transforms.Normalize([0.1635], [0.2076])]
          )
          self.CH3Transforms = transforms.Compose([
                transforms.Grayscale(),
                transforms.CenterCrop(input_size),
                transforms.ToTensor(),
                transforms.Normalize([0.1613], [0.2128])]
          )
          self.CH4Transforms = transforms.Compose([
                transforms.Grayscale(),
                transforms.CenterCrop(input_size),
                transforms.ToTensor(),
                transforms.Normalize([0.1683], [0.2181])]
          )

  def __getitem__(self, index):
      sa_tensor = None
      ch2_tensor = None
      ch3_tensor = None
      ch4_tensor = None
      # for dir in os.listdir(self.patients[index]):
      #   print(os.listdir(self.patients[index]))
      #   if 'SA' in dir:
      #     subDir = os.path.join(self.patients[index], dir)
      #     for file in os.listdir(subDir):
      #       sa_tensor = Image.open(os.path.join(subDir, file))
      #       sa_tensor = self.SATransforms(sa_tensor)
      #   if 'CH2' in dir:
      #     subDir = os.path.join(self.patients[index], dir)
      #     for file in os.listdir(subDir):
      #       ch2_tensor = Image.open(os.path.join(subDir, file))
      #       ch2_tensor = self.CH2Transforms(ch2_tensor)
      #   if 'CH3' in dir:
      #     subDir = os.path.join(self.patients[index], dir)
      #     for file in os.listdir(subDir):
      #       ch3_tensor = Image.open(os.path.join(subDir, file))
      #       ch3_tensor = self.CH3Transforms(ch3_tensor)
      #   if 'CH4' in dir:
      #     subDir = os.path.join(self.patients[index], dir)
      #     for file in os.listdir(subDir):
      #       ch4_tensor = Image.open(os.path.join(subDir, file))
      #       ch4_tensor = self.CH4Transforms(ch4_tensor)
      # print('Started os walk')
      saFolder = os.path.join(self.patients[index], 'SA')
      sa_tensor = Image.open(os.path.join(saFolder, os.listdir(saFolder)[0]))
      sa_tensor = self.SATransforms(sa_tensor)
      ch2Folder = os.path.join(self.patients[index], 'CH2')
      ch2_tensor = Image.open(os.path.join(ch2Folder, os.listdir(ch2Folder)[0]))
      ch2_tensor = self.CH2Transforms(ch2_tensor)
      ch3Folder = os.path.join(self.patients[index], 'CH3')
      ch3_tensor = Image.open(os.path.join(ch3Folder, os.listdir(ch3Folder)[0]))
      ch3_tensor = self.CH3Transforms(ch3_tensor)
      ch4Folder = os.path.join(self.patients[index], 'CH4')
      ch4_tensor = Image.open(os.path.join(ch4Folder, os.listdir(ch4Folder)[0]))
      ch4_tensor = self.CH4Transforms(ch4_tensor)
      # print('Finished os walk, starting tensor build')
      # catTensor = torch.empty(4, sa_tensor.shape[0], sa_tensor.shape[1], sa_tensor.shape[2])
      # catTensor[0] = sa_tensor
      # catTensor[1] = ch2_tensor
      # catTensor[2] = ch3_tensor
      # catTensor[3] = ch4_tensor
      catArray = [sa_tensor, ch2_tensor, ch3_tensor, ch4_tensor]
      # print('Finished tensor build')
      return catArray, self.classIdx[index]

  def __len__(self):
    return len(self.patients)



Defining autoencoder:

In [ ]:
class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 24, 5, padding=2),
            nn.LeakyReLU(0.05, inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(24, 12, 3, padding=1),
            nn.LeakyReLU(0.05, inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(12, 6, 3, padding=1),
            nn.LeakyReLU(0.05, inplace=True),
            nn.MaxPool2d(2)
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(6, 6, 3, padding=1),
            nn.LeakyReLU(0.05, inplace=True),
            nn.UpsamplingBilinear2d(scale_factor=2),
            nn.ConvTranspose2d(6, 12, 3, padding=1),
            nn.LeakyReLU(0.05, inplace=True),
            nn.UpsamplingBilinear2d(scale_factor=2),
            nn.ConvTranspose2d(12, 24, 3, padding=1),
            nn.LeakyReLU(0.05, inplace=True),
            nn.UpsamplingBilinear2d(scale_factor=2),
            nn.ConvTranspose2d(24, 1, 5, padding=2),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

Defining ensemble model:

In [ ]:
class AxisEnsemble(nn.Module):
  def __init__(self, num_of_classes, class_SA_model, class_CH2_model,
               class_CH3_model, class_CH4_model, AEModel_SA, AEModel_CH2,
               AEModel_CH3, AEModel_CH4, device):
    super(AxisEnsemble, self).__init__()
    self.class_SA_model = class_SA_model
    self.class_CH2_model = class_CH2_model
    self.class_CH3_model = class_CH3_model
    self.class_CH4_model = class_CH4_model
    self.device = device
    self.AEModel_SA = AEModel_SA
    self.AEModel_CH2 = AEModel_CH2
    self.AEModel_CH3 = AEModel_CH3
    self.AEModel_CH4 = AEModel_CH4
    self.classifier = nn.Linear(num_of_classes * 4, num_of_classes)

  def forward(self, x1, x2, x3, x4):
    x1_extracted = self.AEModel_SA(x1)
    x2_extracted = self.AEModel_CH2(x2)
    x3_extracted = self.AEModel_CH3(x3)
    x4_extracted = self.AEModel_CH4(x4)
    x1 = torch.abs(x1 - x1_extracted)
    x2 = torch.abs(x2 - x2_extracted)
    x3 = torch.abs(x3 - x3_extracted)
    x4 = torch.abs(x4 - x4_extracted)
    x1_rgb = torch.empty(x1.shape[0], 3, x1.shape[2], x1.shape[3])
    x2_rgb = torch.empty(x2.shape[0], 3, x2.shape[2], x2.shape[3])
    x3_rgb = torch.empty(x3.shape[0], 3, x3.shape[2], x3.shape[3])
    x4_rgb = torch.empty(x3.shape[0], 3, x4.shape[2], x4.shape[3])
    x1_rgb[:] = x1
    x2_rgb[:] = x2
    x3_rgb[:] = x3
    x4_rgb[:] = x4
    x1_rgb = x1_rgb.to(self.device)
    x2_rgb = x2_rgb.to(self.device)
    x3_rgb = x3_rgb.to(self.device)
    x4_rgb = x4_rgb.to(self.device)
    x1_out = self.class_SA_model(x1_rgb)
    x2_out = self.class_CH2_model(x2_rgb)
    x3_out = self.class_CH3_model(x3_rgb)
    x4_out = self.class_CH4_model(x4_rgb)
    x = torch.cat((x1_out, x2_out, x3_out, x4_out), dim=1)
    x = self.classifier(F.relu(x))
    return x

    

In [ ]:
def set_seed():
  seed = 20
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  # torch.backends.cudnn.deterministic = True
  # torch.backends.cudnn.benchmark = False
  np.random.seed(seed)
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)

Helper function for model intialization:

In [ ]:
def initialize_model(num_classes, freeze_percent, dropout):
    model = models.resnet18(pretrained=True, progress=True)
    num_features = model.fc.in_features
    model.fc = nn.Sequential(
       nn.Linear(num_features, num_classes)
    )
    model.layer1.register_forward_hook(lambda m, inp, out: F.dropout(out, p=dropout, training=m.training))
    model.layer3.register_forward_hook(lambda m, inp, out: F.dropout(out, p=dropout, training=m.training))
    # children = model.children()
    # child_cntr = 0
    # for child in children:
    #   child_cntr += 1
    #   if child_cntr == 5 or child_cntr == 7:
    #     for param in child.parameters():
    #       param.requires_grad = False
    #   if child_cntr > 7:
    #     break

    params_to_update = model.parameters()

    param_cnt = 0
    for params in params_to_update:
      param_cnt += 1
    cur_param_cnt = 0
    params_to_update = model.parameters()
    for params in params_to_update:
      if cur_param_cnt > param_cnt*freeze_percent:
        param.requires_grad = False
      cur_param_cnt += 1  

    return model

In [ ]:
def tune_fnc(config):
  data_dir = '/content/drive/My Drive/BME/7felev/Szakdolgozat/patients4'

  batch_size = config["batch_size"]

  num_epochs = 80

  input_size = 168

  AEmodel_SA = AE()
  AEmodel_SA.load_state_dict(torch.load("/content/drive/My Drive/BME/7felev/Szakdolgozat/AEWeights/new/sa_aeweights"))
  AEmodel_SA.eval()
  for param in AEmodel_SA.parameters():
      param.requires_grad = False
  AEmodel_CH2 = AE()
  AEmodel_CH2.load_state_dict(torch.load("/content/drive/My Drive/BME/7felev/Szakdolgozat/AEWeights/new/ch2_aeweights"))
  AEmodel_CH2.eval()
  for param in AEmodel_CH2.parameters():
      param.requires_grad = False
  AEmodel_CH3 = AE()
  AEmodel_CH3.load_state_dict(torch.load("/content/drive/My Drive/BME/7felev/Szakdolgozat/AEWeights/new/ch3_aeweights"))
  AEmodel_CH3.eval()
  for param in AEmodel_CH3.parameters():
      param.requires_grad = False
  AEmodel_CH4 = AE()
  AEmodel_CH4.load_state_dict(torch.load("/content/drive/My Drive/BME/7felev/Szakdolgozat/AEWeights/new/ch4_aeweights"))
  AEmodel_CH4.eval()
  for param in AEmodel_CH4.parameters():
      param.requires_grad = False

  print("Initializing Datasets and Dataloaders...")

  # def make_weights_for_balanced_classes(images, nclasses):                        
  #     count = [0] * nclasses                                                      
  #     for item in images:                                                         
  #         count[item[1]] += 1                                                     
  #     weight_per_class = [0.] * nclasses                                      
  #     N = float(sum(count))                                                   
  #     for i in range(nclasses):                                                   
  #         weight_per_class[i] = N/float(count[i])                                 
  #     weight = [0] * len(images)                                              
  #     for idx, val in enumerate(images):                                          
  #         weight[idx] = weight_per_class[val[1]]                                  
  #     return weight

  # Create training and validation datasets
  patient_datasets = {x: PatientDataset(os.path.join(data_dir, x), input_size, x) for x in ['training', 'validation', 'test']}
  # trainWeights = make_weights_for_balanced_classes(patient_datasets['training'].patientSamples, len(patient_datasets['training'].classes))
  # trainWeights = torch.DoubleTensor(trainWeights)
  # trainSampler = torch.utils.data.sampler.WeightedRandomSampler(trainWeights, len(trainWeights))

  # Create training and validation dataloaders
  trainDataLoader = torch.utils.data.DataLoader(patient_datasets['training'], batch_size=batch_size, shuffle=True, num_workers=2)
  valDataLoader = torch.utils.data.DataLoader(patient_datasets['validation'], batch_size=batch_size, shuffle=True, num_workers=2)
  testDataLoader = torch.utils.data.DataLoader(patient_datasets['test'], batch_size=batch_size, shuffle=True, num_workers=2)
  dataloaders_dict = {'training': trainDataLoader, 'validation': valDataLoader }

  # Detect if we have a GPU available
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  num_classes = len(patient_datasets['training'].classes)
  set_seed()
  SA_subModel = initialize_model(num_classes, config["freeze_percent"], config["dropout"])
  set_seed()
  CH2_subModel = initialize_model(num_classes, config["freeze_percent"], config["dropout"])
  set_seed()
  CH3_subModel = initialize_model(num_classes, config["freeze_percent"], config["dropout"])
  set_seed()
  CH4_subModel = initialize_model(num_classes, config["freeze_percent"], config["dropout"])
  set_seed()
  ensembleModel = AxisEnsemble(num_classes, SA_subModel, CH2_subModel,
                              CH3_subModel, CH4_subModel, AEmodel_SA,
                              AEmodel_CH2, AEmodel_CH3, AEmodel_CH4, device)


  # Send the model to GPU
  ensembleModel = ensembleModel.to(device)

  params_to_update = ensembleModel.parameters()

  optimizer = None

  if (config["is_amsgrad"] == 1):
    optimizer = optim.Adam(params_to_update, lr=config["lr"], weight_decay=config["wd"], amsgrad=True)
  else:
    optimizer = optim.Adam(params_to_update, lr=config["lr"], weight_decay=config["wd"])
  # optimizer = optim.Adam(params_to_update, lr=0.0001)
  # scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.85)
  scheduler = None

  weights = torch.Tensor(patient_datasets['training'].weightsPerClasses).to(device)
  criterion = nn.CrossEntropyLoss(weights)
  # criterion = nn.CrossEntropyLoss()

  dataloaders = dataloaders_dict
  classModel = ensembleModel

  set_seed()

  since = time.time()

  val_acc_history = []
  train_acc_history = []

  val_f1_history = []
  train_f1_history = []

  val_loss_history = []
  train_loss_history = []

  best_model_wts = copy.deepcopy(classModel.state_dict())
  best_overall_loss = 10000.0
  acc_at_best_loss = 0.0
  f1_at_best_loss = 0.0
  loss_at_best_acc = 10000.0
  f1_at_best_acc = 0.0
  loss_at_best_f1 = 10000.0
  acc_at_best_f1 = 0.0
  best_acc = 0.0
  best_f1 = 0.0
  best_loss = 10000.0
  best_loss_preds, best_loss_targets = [], []
  best_f1_preds, best_f1_targets = [], []
  best_acc_preds, best_acc_targets = [], []
  training_acc_at_best_val_loss = 0.0
  training_f1_at_best_val_loss = 0.0
  training_loss_at_best_val_loss = 10000.0
  training_acc_at_best_val_acc = 0.0
  training_f1_at_best_val_acc = 0.0
  training_loss_at_best_val_acc = 10000.0
  training_acc_at_best_val_f1 = 0.0
  training_f1_at_best_val_f1 = 0.0
  training_loss_at_best_val_f1 = 10000.0

  val_f1_acc = 0.0

  epochs_without_upgrade = 0

  for epoch in range(num_epochs):
      print('Epoch {}/{}'.format(epoch, num_epochs - 1))
      print('-' * 10)

      prev_train_loss = 10000.0

      # Each epoch has a training and validation phase
      for phase in ['training', 'validation']:
          if phase == 'training':
              classModel.train()  # Set model to training mode
          else:
              classModel.eval()   # Set model to evaluate mode

          running_loss = 0.0
          running_corrects = 0

          tmp_preds, tmp_targets = [], []
          tmp_preds_arr, tmp_targets_arr = [], []

          # Iterate over data.
          for inputs, labels in dataloaders[phase]:
              # print('Started batch')
              saInputs, ch2Inputs, ch3Inputs, ch4Inputs = inputs
              saInputs = saInputs.to(device)
              ch2Inputs = ch2Inputs.to(device)
              ch3Inputs = ch3Inputs.to(device)
              ch4Inputs = ch4Inputs.to(device)
              labels = labels.to(device)

              # zero the parameter gradients
              optimizer.zero_grad()

              # forward
              # track history if only in train
              with torch.set_grad_enabled(phase == 'training'):
                  # Get model outputs and calculate loss
                  outputs = classModel(saInputs, ch2Inputs, ch3Inputs, ch4Inputs)

                  # g = make_dot(outputs)
                  # g.view()

                  loss = criterion(outputs, labels)

                  _, preds = torch.max(outputs, 1)
                  # print('Predictions: ')
                  # print(preds)

                  # backward + optimize only if in training phase
                  if phase == 'training':
                      # #L1 regularization
                      # all_weights = []
                      # for name, param in classModel.named_parameters():
                      #   if 'weight' in name:
                      #     all_weights.append(param.view(-1))
                      # # print('weight num: {}'.format(len(all_weights)))
                      # all_weights_params = torch.cat(all_weights)
                      # l1_regularization = 0.001 * torch.norm(all_weights_params, 1)

                      # loss = loss + l1_regularization

                      loss.backward()
                      optimizer.step()
                      # loss = loss - l1_regularization

              # statistics
              running_loss += loss.item() * inputs[0].size(0)
              running_corrects += torch.sum(preds == labels)

              detached_pred = preds.detach().cpu().numpy()
              detached_target = labels.detach().cpu().numpy()

              tmp_preds_arr.append(detached_pred)
              tmp_targets_arr.append(detached_target)

              for i in range(detached_pred.shape[0]):
                  tmp_preds.append(detached_pred[i])
                  tmp_targets.append(detached_target[i])
            

          epoch_loss = running_loss / len(dataloaders[phase].sampler)
          epoch_acc = torch.true_divide(running_corrects, len(dataloaders[phase].sampler))
          epoch_f1 = f1_score(np.concatenate(tmp_preds_arr), np.concatenate(tmp_targets_arr), average="macro")

          print('{} Loss: {:.4f} Acc: {:.4f} F1: {:.4f}'.format(phase, epoch_loss, epoch_acc, epoch_f1))

          # deep copy the model
          if phase == 'validation':
              val_acc_history.append(epoch_acc)
              val_loss_history.append(epoch_loss)
              val_f1_history.append(epoch_f1)

              if best_overall_loss > epoch_loss + prev_train_loss:
                best_overall_loss = epoch_loss + prev_train_loss
                loss_at_best_loss = epoch_loss
                acc_at_best_loss = epoch_acc
                f1_at_best_loss = epoch_f1
                training_acc_at_best_val_loss = train_acc_history[len(train_acc_history) - 1]
                training_f1_at_best_val_loss = train_f1_history[len(train_f1_history) - 1]
                training_loss_at_best_val_loss = train_loss_history[len(train_loss_history) - 1]
                best_loss_preds = tmp_preds
                best_loss_targets = tmp_targets
              if best_loss > epoch_loss:
                best_loss = epoch_loss
                epochs_without_upgrade = 0 
              if best_acc < epoch_acc:
                best_acc = epoch_acc
                best_acc_preds = tmp_preds
                best_acc_targets = tmp_targets
                loss_at_best_acc = epoch_loss
                f1_at_best_acc = epoch_f1
                training_acc_at_best_val_acc = train_acc_history[len(train_acc_history) - 1]
                training_f1_at_best_val_acc = train_f1_history[len(train_f1_history) - 1]
                training_loss_at_best_val_acc = train_loss_history[len(train_loss_history) - 1]
                epochs_without_upgrade = 0
              if best_f1 < epoch_f1:
                best_f1 = epoch_f1 
                best_f1_preds = tmp_preds
                best_f1_targets = tmp_targets
                loss_at_best_f1 = epoch_loss
                acc_at_best_f1 = epoch_acc
                training_acc_at_best_val_f1 = train_acc_history[len(train_acc_history) - 1]
                training_f1_at_best_val_f1 = train_f1_history[len(train_f1_history) - 1]
                training_loss_at_best_val_f1 = train_loss_history[len(train_loss_history) - 1]
                best_model_wts = copy.deepcopy(classModel.state_dict())
                epochs_without_upgrade = 0

              # scheduler.step()
          if phase == 'training':
              prev_train_loss = epoch_loss
              train_acc_history.append(epoch_acc)
              train_f1_history.append(epoch_f1)
              train_loss_history.append(epoch_loss)
      print()        
      # if (epochs_without_upgrade > 19):
      #   print("Terminating training due to no convergence.")
      #   break
      # epochs_without_upgrade += 1

      tune.report(val_loss=val_loss_history[len(val_loss_history) - 1],
                  val_acc=val_acc_history[len(val_acc_history) - 1],
                  val_f1=val_f1_history[len(val_f1_history) - 1],
                  train_loss=train_loss_history[len(train_loss_history) - 1],
                  train_acc=train_acc_history[len(train_acc_history) - 1],
                  train_f1=train_f1_history[len(train_f1_history) - 1],
                  training_iteration=epoch,
                  best_val_f1=best_f1,
                  val_acc_at_best_f1=acc_at_best_f1,
                  best_val_acc=best_acc,
                  val_f1_at_best_acc=f1_at_best_acc,
                  val_f1_acc=val_acc_history[len(val_acc_history) - 1] + val_f1_history[len(val_f1_history) - 1])


Starting the training and tuning, displaying metrics:

In [ ]:
config = {
        "lr": tune.loguniform(1e-5, 1e-4),
        "wd": tune.loguniform(1e-5, 1e-4),
        "batch_size": tune.choice([4, 8, 16]),
        "freeze_percent": tune.choice([50, 70, 100]),
        "dropout": tune.choice([0, 0.2]),
        "is_amsgrad": tune.choice([0, 1])
}

scheduler = ASHAScheduler(
        metric="val_f1_acc",
        mode="max",
        max_t=20,
        grace_period=7,
        reduction_factor=3)

reporter = CLIReporter(
        parameter_columns=["lr", "wd", "batch_size", "freeze_percent", "dropout", "is_amsgrad"],
        metric_columns=["val_loss", "val_acc", "val_f1", "val_f1_acc", "training_iteration",
                        "train_loss", "train_acc", "train_f1", "best_val_acc",
                        "val_f1_at_best_acc", "best_val_f1", "val_acc_at_best_f1"])

result = tune.run(
        tune_fnc,
        resources_per_trial={"cpu": 1, "gpu": 1},
        config=config,
        num_samples=12,
        scheduler=scheduler,
        progress_reporter=reporter)


(pid=916) 2021-05-16 12:57:06,370	ERROR worker.py:382 -- SystemExit was raised from the worker
(pid=916) Traceback (most recent call last):
(pid=916)   File "python/ray/_raylet.pyx", line 599, in ray._raylet.task_execution_handler
(pid=916)   File "python/ray/_raylet.pyx", line 451, in ray._raylet.execute_task
(pid=916)   File "python/ray/_raylet.pyx", line 488, in ray._raylet.execute_task
(pid=916)   File "python/ray/_raylet.pyx", line 495, in ray._raylet.execute_task
(pid=916)   File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
(pid=916)   File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_executor
(pid=916)   File "/usr/local/lib/python3.7/dist-packages/ray/_private/function_manager.py", line 556, in actor_method_executor
(pid=916)     return method(__ray_actor, *args, **kwargs)
(pid=916)   File "/usr/local/lib/python3.7/dist-packages/ray/tune/trainable.py", line 173, in train_buffered
(pid=916)     result = self.train()
(pid=916)   File

== Status ==
Memory usage on this node: 3.0/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 7.000: None
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.32 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/tune_fnc_2021-05-16_12-57-06
Number of trials: 12/12 (12 PENDING)
+----------------------+----------+-------+-------------+-------------+--------------+------------------+-----------+--------------+
| Trial name           | status   | loc   |          lr |          wd |   batch_size |   freeze_percent |   dropout |   is_amsgrad |
|----------------------+----------+-------+-------------+-------------+--------------+------------------+-----------+--------------|
| tune_fnc_37b05_00000 | PENDING  |       | 4.91988e-05 | 3.44998e-05 |            4 |              100 |       0.2 |            1 |
| tune_fnc_37b05_00001 | PENDING  |       | 3.61003e-05 | 1.33154e-05 |           16 |              100 |       0   |            1 |
| tune_fnc

(raylet) /usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)


== Status ==
Memory usage on this node: 2.7/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 7.000: None
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.32 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/tune_fnc_2021-05-16_12-57-06
Number of trials: 12/12 (11 PENDING, 1 RUNNING)
+----------------------+----------+-------+-------------+-------------+--------------+------------------+-----------+--------------+
| Trial name           | status   | loc   |          lr |          wd |   batch_size |   freeze_percent |   dropout |   is_amsgrad |
|----------------------+----------+-------+-------------+-------------+--------------+------------------+-----------+--------------|
| tune_fnc_37b05_00000 | RUNNING  |       | 4.91988e-05 | 3.44998e-05 |            4 |              100 |       0.2 |            1 |
| tune_fnc_37b05_00001 | PENDING  |       | 3.61003e-05 | 1.33154e-05 |           16 |              100 |       0   |           

2021-05-16 12:57:40,625	WARNING tune.py:507 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


KeyboardInterrupt: ignored